In [172]:
import os
from collections import Counter
from itertools import product
from tqdm import tqdm
import torch
import math
import random

In [173]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [174]:


folder_path = 'dat410_europarl/'

translations = {}
file_list = os.listdir(folder_path)
text_files = [file for file in file_list]
for text_file in text_files:
    file_path = os.path.join(folder_path, text_file)
    with open(file_path, 'r', encoding='utf-8') as file:
        translations[file_path.split(".")[-1]] = [l[:-1] for l in file.readlines()]
        
translations["sv"]

['jag förklarar europaparlamentets session återupptagen efter avbrottet den 17 december . jag vill på nytt önska er ett gott nytt år och jag hoppas att ni haft en trevlig semester .',
 'som ni kunnat konstatera ägde &quot; den stora år 2000-buggen &quot; aldrig rum . däremot har invånarna i ett antal av våra medlemsländer drabbats av naturkatastrofer som verkligen varit förskräckliga .',
 'ni har begärt en debatt i ämnet under sammanträdesperiodens kommande dagar .',
 'till dess vill jag att vi , som ett antal kolleger begärt , håller en tyst minut för offren för bl.a. stormarna i de länder i europeiska unionen som drabbats .',
 'jag ber er resa er för en tyst minut .',
 'ni känner till från media att det skett en rad bombexplosioner och mord i sri lanka .',
 'en av de personer som mycket nyligen mördades i sri lanka var kumar ponnambalam , som besökte europaparlamentet för bara några månader sedan .',
 'skulle det vara möjligt för er , fru talman , att skriva ett brev till den srilank

In [175]:
simpel_translator = torch.load("simpel_translator.pth")

In [176]:
translator_dict = {}
for (swe_word, eng_word), prob in simpel_translator.items():
    eng_dict = translator_dict.get(eng_word, {})
    eng_dict[swe_word] = prob
    translator_dict[eng_word] = eng_dict


In [177]:

def simpel_translate_setance(sentence, translator_dict):
    out = []
    for word in sentence.split(" "):
        out.append(max(translator_dict[word], key=lambda k: translator_dict[word][k]))
    return " ".join(out)
sentence = "mrs lynne , you are quite right and i shall check whether this has actually not been done ."

simpel_translate_setance(sentence, translator_dict)

'. lynne , ni är helt rätt och jag kommer kontrollera om detta har verkligen inte har gjort .'

In [178]:
swe_count = Counter(" ".join(translations["sv"]).split(" "))
eng_count = Counter(" ".join(translations["en"]).split(" "))
print(swe_count.most_common(20))
print(eng_count.most_common(20))

[('.', 9648), ('att', 9181), (',', 8876), ('och', 7038), ('i', 5949), ('det', 5687), ('som', 5028), ('för', 4959), ('av', 4013), ('är', 3840), ('en', 3724), ('vi', 3211), ('jag', 3093), ('den', 2953), ('de', 2930), ('om', 2909), ('till', 2837), ('på', 2783), ('har', 2484), ('inte', 2385)]
[('the', 19322), (',', 13514), ('.', 9774), ('of', 9312), ('to', 8801), ('and', 6946), ('in', 6090), ('is', 4400), ('that', 4357), ('a', 4269), ('we', 3223), ('this', 3222), ('i', 2964), ('for', 2942), ('be', 2550), ('it', 2426), ('on', 2304), ('which', 2026), ('are', 1917), ('have', 1881)]


In [179]:
eng_vocab = [t[0] for t in eng_count.most_common()]
eng_vocab = ["<UKN>", "<PAD>", "<s>", "</s>"] + eng_vocab[:int(len(eng_vocab) * 0.30)]
print("Portion of unknow english words in the corpus", 1 -sum([eng_count[w] for w in eng_vocab]) / sum(eng_count.values()))
print("Length of vocab",  len(eng_vocab))

Portion of unknow english words in the corpus 0.051372053034717036
Length of vocab 3334


In [180]:
eng_id2word = {i:w for i, w in enumerate(eng_vocab)}
eng_word2id = {w:i for i, w in enumerate(eng_vocab)}

In [181]:
#Transalte wprds in to ids, with start and end tokens at respective ends
eng_tokens = [[2] + [eng_word2id.get(w, 0) for w in scent.split(" ")] + [3] for scent in  translations["en"]]
max_len = max([len(ids) for ids in eng_tokens])
#Add padding such that all scentances are of equal length
eng_tokens = [scent + [1] * (max_len - len(scent)) for scent in eng_tokens]
#Assert that they are of equal length
assert all([len(ids) == len(eng_tokens[0]) for ids in eng_tokens])
eng_tokens, len(eng_tokens), len(eng_tokens[0])

([[2,
   16,
   2987,
   2737,
   4,
   2043,
   7,
   4,
   29,
   49,
   0,
   20,
   2338,
   2181,
   1030,
   243,
   5,
   9,
   16,
   34,
   54,
   288,
   225,
   8,
   388,
   41,
   13,
   1153,
   82,
   260,
   10,
   4,
   217,
   12,
   41,
   0,
   13,
   0,
   0,
   342,
   6,
   3,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   

In [182]:
swe_vocab = [t[0] for t in swe_count.most_common()]
swe_vocab = ["<UKN>", "<PAD>", "<s>", "</s>"] + swe_vocab[:int(len(swe_vocab) * 0.35)]
print("Portion of unknow swedish words in the corpus", 1 -sum([swe_count[w] for w in swe_vocab]) / sum(swe_count.values()))
print("Length of vocab",  len(swe_vocab))

Portion of unknow swedish words in the corpus 0.05783098983711543
Length of vocab 6441


In [183]:
swe_id2word = {i:w for i, w in enumerate(swe_vocab)}
swe_word2id = {w:i for i, w in enumerate(swe_vocab)}

In [184]:
#Transalte wprds in to ids, with start and end tokens at respective ends
swe_tokens = [[2] + [swe_word2id.get(w, 0) for w in scent.split(" ")] + [3] for scent in  translations["sv"]]
max_len = max([len(ids) for ids in swe_tokens])
#Add padding such that all scentances are of equal length
swe_tokens = [scent + [1] * (max_len - len(scent)) for scent in swe_tokens]
#Assert that they are of equal length
assert all([len(ids) == len(swe_tokens[0]) for ids in swe_tokens])
swe_tokens, len(swe_tokens), len(swe_tokens[0])

([[2,
   16,
   1553,
   424,
   3535,
   4943,
   127,
   4141,
   17,
   2101,
   969,
   4,
   16,
   37,
   21,
   311,
   2102,
   94,
   25,
   913,
   311,
   99,
   7,
   16,
   183,
   5,
   53,
   508,
   14,
   6285,
   6286,
   4,
   3,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
 

In [193]:
def random_split_list(input_list, split_ratio):
    assert 0 <= split_ratio <= 1, "Split ratio must be between 0 and 1"

    values = input_list.copy()
    random.shuffle(values)

    split_index = int(len(values) * split_ratio)
    first_list = values[:split_index]
    second_list = values[split_index:]

    return first_list, second_list

In [195]:
train_indecies, rest = random_split_list(list(range(len(eng_tokens))), 0.7)
val_indecies, test_indecies= random_split_list(rest, 0.5)
assert len(train_indecies) + len(val_indecies) + len(test_indecies) == len(eng_tokens)
assert sorted(train_indecies + val_indecies + test_indecies) == list(range(len(eng_tokens)))
train_indecies[:10], val_indecies[:10], test_indecies[:10]

([1056, 6205, 4153, 750, 3279, 5565, 6748, 2959, 4219, 6495],
 [7358, 6671, 3355, 7007, 3291, 4357, 728, 5594, 8527, 6124],
 [1372, 6772, 8055, 6196, 6133, 8864, 6226, 6401, 8105, 2609])

In [198]:
train_eng = torch.tensor(eng_tokens)[train_indecies].to(device)
train_swe = torch.tensor(swe_tokens)[train_indecies].to(device)

val_eng = torch.tensor(eng_tokens)[val_indecies].to(device)
val_swe = torch.tensor(swe_tokens)[val_indecies].to(device)

test_eng = torch.tensor(eng_tokens)[test_indecies].to(device)
test_swe = torch.tensor(swe_tokens)[test_indecies].to(device)

assert train_eng.shape[0] + val_eng.shape[0] + test_eng.shape[0] == len(eng_tokens)
train_eng.shape

torch.Size([7000, 187])